In [ ]:


# Установка зависимостей
!apt-get -y install libssl-dev libffi-dev
!pip install -q pygit2==1.15.1 cffi gdown
!pip install -q torch torchvision torchaudio transformers==4.42.4 torchsde --index-url https://download.pytorch.org/whl/cu121

# Очистка пакетов и кэша
!apt-get clean
!rm -rf /var/lib/apt/lists/*

# Клонируем Foocus
%cd /content
!git clone https://github.com/lllyasviel/Fooocus.git

# Создаем папки для моделей и LoRA
!mkdir -p /content/Fooocus/models/checkpoints
!mkdir -p /content/Fooocus/models/lora

# Скачиваем модель CyberRealisticPony с Hugging Face
!wget "https://huggingface.co/cyberdelia/CyberRealisticPony/resolve/main/CyberRealisticPony_V10.0_FP16.safetensors?download=true" \
    -O /content/Fooocus/models/checkpoints/CyberRealisticPonyXL_fp16.safetensors

# Скачиваем только одну модель perfectionAmateurILXL_v20 с Google Drive (оставь только нужную!)
!gdown "1D-TgTwIOhtPDgsea7H7BbMk3hCNjHMri" \
    -O /content/Fooocus/models/checkpoints/perfectionAmateurILXL_v20.safetensors

# Скачиваем LoRA с Hugging Face
!wget "https://huggingface.co/embed/lora/resolve/main/animeLineartMangaLike_v30.safetensors" \
    -O /content/Fooocus/models/lora/animeLineartMangaLike.safetensors

# Настроим Foocus, чтобы работала только одна выбранная модель (минимум памяти)
import json
config_path = "/content/Fooocus/config.json"
config = {
    "default_checkpoint": "perfectionAmateurILXL_v20.safetensors",
    "default_lora": ["animeLineartMangaLike.safetensors"]
}
with open(config_path, "w") as f:
    json.dump(config, f, indent=4)

# Максимальная очистка памяти перед запуском Foocus
import gc
import torch
gc.collect()
torch.cuda.empty_cache()

# Переходим в папку Foocus и запускаем Foocus в lowvram режиме
%cd /content/Fooocus
!python entry_with_update.py --share --always-high-vram